<a href="https://colab.research.google.com/github/JD-Lora1/LLM-RAG-youtube-transcript/blob/main/RAG_IA2_InvarRAG_way.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Instalar las librerías necesarias.
!pip install -q chromadb sentence-transformers youtube-transcript-api transformers accelerate bitsandbytes
!pip install -q torch
!pip install -q einops

# --- 1. Importar Librerías ---
from youtube_transcript_api import YouTubeTranscriptApi
import chromadb
from chromadb.utils import embedding_functions
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch

# --- 2. Extracción de la Transcripción y Almacenamiento en ChromaDB ---

# Video de ejemplo
video_id = "e1mOmdykmwI"

# Usamos tu método de API, que funciona en tu entorno
ytt_api = YouTubeTranscriptApi()

try:
    transcript = ytt_api.fetch(video_id)
    print("Transcripción extraída exitosamente.")
except Exception as e:
    print(f"Error al extraer la transcripción: {e}")
    transcript = []

if not transcript:
    print("No hay transcripción para procesar. Saliendo.")
else:
    # Procesar la transcripción y extraer textos y metadatos
    texts = []
    metadatas = []
    for i, entry in enumerate(transcript):
        texts.append(entry.text)
        metadatas.append({
            'start': entry.start,
            'duration': entry.duration,
            'order': i
        })

    # Inicializar la función de embedding
    sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(
        model_name="all-MiniLM-L6-v2"
    )

    # Crear el cliente de ChromaDB
    chroma_client = chromadb.Client()

    # Corregido: Manejo de errores más robusto al eliminar la colección
    try:
        chroma_client.delete_collection(name="youtube_transcripts")
        print("Colección existente eliminada para empezar de cero.")
    except Exception as e:
        print(f"No se pudo eliminar la colección (posiblemente porque no existía). Continuando...")
        pass

    # Crear una colección con la función de embedding
    collection = chroma_client.create_collection(
        name="youtube_transcripts",
        embedding_function=sentence_transformer_ef
    )

    # Generar IDs únicos para cada segmento
    ids = [str(i) for i in range(len(texts))]

    # Añadir a la colección
    collection.add(
        documents=texts,
        metadatas=metadatas,
        ids=ids
    )
    print("Documentos almacenados en ChromaDB.")

    # --- 3. Implementación del Pipeline RAG con LLM Local ---

    # Cargar el modelo Qwen2-0.5B-Instruct y el tokenizador
    model_name = "Qwen/Qwen2-0.5B-Instruct"
    try:
        print(f"\nCargando el modelo {model_name}...")
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModelForCausalLM.from_pretrained(
            model_name,
            device_map="auto",
            torch_dtype=torch.float16
        )
        # Configurar el pipeline de generación de texto
        pipe = pipeline(
            "text-generation",
            model=model,
            tokenizer=tokenizer,
            max_new_tokens=256,
            do_sample=True,
            temperature=0.7,
            top_k=50
        )
        print("Modelo cargado exitosamente.")
    except Exception as e:
        print(f"Error al cargar el modelo. ¿Tienes suficiente VRAM en la GPU? {e}")
        pipe = None

    if pipe:
        # Función para el pipeline completo de RAG
        def rag_pipeline_qwen(query: str, collection, pipe, n_results: int = 5):
            # A) Recuperación: Buscar documentos relevantes en ChromaDB
            retrieved_docs = collection.query(
                query_texts=[query],
                n_results=n_results
            )['documents'][0]

            # B) Aumentación: Construir el prompt con el contexto recuperado
            context = "\n".join(retrieved_docs)
            messages = [
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": f"Based on the following context, answer the question. If the answer is not in the context, state that you don't have the information. \n\nContext:\n{context}\n\nQuestion: {query}"}
            ]

            # C) Generación: Usar el LLM para generar una respuesta
            prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
            outputs = pipe(prompt)
            # Extraer solo la respuesta del modelo, sin el prompt
            generated_response = outputs[0]['generated_text'][len(prompt):]

            return generated_response

        # Ejemplo de uso del pipeline RAG
        user_query = "De qué trata el video?"
        print(f"\n--- Pregunta al RAG: '{user_query}' ---")
        final_response = rag_pipeline_qwen(user_query, collection, pipe)
        print(f"--- Respuesta del LLM (Qwen) ---\n{final_response}")

Transcripción extraída exitosamente.
Colección existente eliminada para empezar de cero.
Documentos almacenados en ChromaDB.

Cargando el modelo Qwen/Qwen2-0.5B-Instruct...


Device set to use cpu


Modelo cargado exitosamente.

--- Pregunta al RAG: 'De qué trata el video?' ---
--- Respuesta del LLM (Qwen) ---
El video trata sobre la conexión entre las emociones y la música.


Transcripción extraída exitosamente.
